In [1]:
import numpy as np
from keras.datasets import mnist
import gc

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD

import cv2
import h5py as h5py 
import numpy as np

Using CNTK backend


In [2]:
def tran_y(y): 
    y_ohe = np.zeros(10) 
    y_ohe[y] = 1 
    return y_ohe

首先将数据整合为VGG所需维度，由于硬件配置限制，我们选用48个像素点而不是原VGG16所采用的224个像素点。即使这样仍然需要24GB以上内存或者使用数据生成器。


In [4]:
# 如果硬件配置较高，比如主机具备32GB以上内存，GPU具备8GB以上显存，可以适当增大这个值。VGG要求至少48像素
ishape=48
(X_train, y_train), (X_test, y_test) = mnist.load_data() 

X_train = [cv2.cvtColor(cv2.resize(i, (ishape, ishape)), cv2.COLOR_GRAY2BGR) for i in X_train] 
X_train = np.concatenate([arr[np.newaxis] for arr in X_train]).astype('float32') 
X_train /= 255.0

X_test = [cv2.cvtColor(cv2.resize(i, (ishape, ishape)), cv2.COLOR_GRAY2BGR) for i in X_test] 
X_test = np.concatenate([arr[np.newaxis] for arr in X_test]).astype('float32')
X_test /= 255.0

y_train_ohe = np.array([tran_y(y_train[i]) for i in range(len(y_train))]) 
y_test_ohe = np.array([tran_y(y_test[i]) for i in range(len(y_test))])
y_train_ohe = y_train_ohe.astype('float32')
y_test_ohe = y_test_ohe.astype('float32')

In [4]:
X_train.shape

(60000, 96, 96, 3)

VGG16 全参重训迁移学习


In [5]:
model_vgg = VGG16(include_top = False, weights = 'imagenet', input_shape = (ishape,ishape,3))
model = Flatten(name = 'flatten')(model_vgg.output) 
model = Dense(4096, activation='relu', name='fc1')(model)
model = Dense(4096, activation='relu', name='fc2')(model)
model = Dropout(0.5)(model)
model = Dense(10, activation = 'softmax')(model) 
model_vgg_mnist = Model(model_vgg.input, model, name = 'vgg16')

model_vgg_mnist.compile(loss = 'categorical_crossentropy', optimizer = 'adagrad', metrics = ['accuracy'])
model_vgg_mnist.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0         
__________

In [6]:
# 2的小批量数保证在一台32GB内存的PC上用一个1060 6GB版本GPU可以运行
model_vgg_mnist.fit(X_train, y_train_ohe, validation_data = (X_test, y_test_ohe), epochs = 2, batch_size = 2)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 1755s - loss: 14.4706 - acc: 0.1022 - val_loss: 14.4902 - val_acc: 0.1010
Epoch 2/2
60000/60000 [==============================] - 1742s - loss: 14.4716 - acc: 0.1022 - val_loss: 14.4902 - val_acc: 0.1010


直接使用VGG16会发现拟合效果非常差。一个解决办法是将卷积层的参数固化，不参与训练。

In [5]:
try:
    del(model_vgg_mnist)
except:
    print('object gone')
# 很多时候需要多次回收垃圾才能彻底收回内存。如果不行，重新启动单独执行下面的模型
for i in range(10):
    gc.collect()

object gone


VGG16 部分参数重训迁移学习

In [8]:
ishape=224
model_vgg = VGG16(include_top = False, weights = 'imagenet', input_shape = (ishape, ishape, 3)) 
#for i, layer in enumerate(model_vgg.layers): 
#    if i<20:
for layer in model_vgg.layers:
        layer.trainable = False
model = Flatten()(model_vgg.output) 
model = Dense(4096, activation='relu', name='fc1')(model)
model = Dense(4096, activation='relu', name='fc2')(model)
model = Dropout(0.5)(model)
model = Dense(10, activation = 'softmax', name='prediction')(model) 
model_vgg_mnist_pretrain = Model(model_vgg.input, model, name = 'vgg16_pretrain')
model_vgg_mnist_pretrain.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
sgd = SGD(lr = 0.05, decay = 1e-5) 
model_vgg_mnist_pretrain.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

In [10]:
model_vgg_mnist_pretrain.fit(X_train, y_train_ohe, validation_data = (X_test, y_test_ohe), epochs = 10, batch_size = 64)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 64s - loss: 0.5499 - acc: 0.8196 - val_loss: 0.3690 - val_acc: 0.8791
Epoch 2/10
60000/60000 [==============================] - 59s - loss: 0.2454 - acc: 0.9211 - val_loss: 0.2055 - val_acc: 0.9364
Epoch 3/10
60000/60000 [==============================] - 60s - loss: 0.1871 - acc: 0.9396 - val_loss: 0.4686 - val_acc: 0.8455
Epoch 4/10
60000/60000 [==============================] - 60s - loss: 0.1559 - acc: 0.9498 - val_loss: 0.1571 - val_acc: 0.9505
Epoch 5/10
60000/60000 [==============================] - 59s - loss: 0.1404 - acc: 0.9540 - val_loss: 0.1388 - val_acc: 0.9542
Epoch 6/10
60000/60000 [==============================] - 59s - loss: 0.1266 - acc: 0.9583 - val_loss: 0.2069 - val_acc: 0.9334
Epoch 7/10
60000/60000 [==============================] - 59s - loss: 0.1193 - acc: 0.9606 - val_loss: 0.1063 - val_acc: 0.9656
Epoch 8/10
60000/60000 [==============================

回收显存和内存垃圾可以使用如下方法，通常需要多次调用gc.collect()才能清除干净。如果异常退出，则无法清除干净显存，需要重新启动notebook。

In [9]:
#del(model_vgg_mnist_pretrain, model_vgg, model)
for i in range(100):
    gc.collect()